# Libraries

In [1]:
%reload_ext autoreload
%autoreload 2
#matplotlib inline

In [2]:
! pip install transformers[sentencepiece] fastbook fastai ohmeow-blurr nbdev

^C



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

ModuleNotFoundError: No module named 'blurr.text'

### cleaning

In [26]:
import pandas as pd

In [27]:
df= pd.read_csv('Dataset(final).csv')
df.head(2)

,Name,Description,Cleaned_recipe,Origin,Unnamed: 4,Non-vegan,Non-Vegetarian,Sea-foods,Haram,Nut-allergies,Non-Pescatarian,pork,Meat,beaf,alcohol,dairy,Restriction,Restrictive Ingredient
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df['Body'] = df['Description'].astype(str) + " " + df['Cleaned_recipe'].astype(str) 
df.head(2)

,Name,Description,Cleaned_recipe,Origin,Unnamed: 4,Non-vegan,Non-Vegetarian,Sea-foods,Haram,Nut-allergies,Non-Pescatarian,pork,Meat,beaf,alcohol,dairy,Restriction,Restrictive Ingredient,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This Amish pickled beets recipe is a gift from...


In [29]:
# Assuming you have a DataFrame named df and you want to drop columns 5 to 17
columns_to_drop = df.columns[4:16]  # This selects columns 5 to 17 (0-based index)

# Use the drop method to remove the specified columns
df = df.drop(columns=columns_to_drop)
df.head(2)

,Name,Description,Cleaned_recipe,Origin,Restriction,Restrictive Ingredient,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,NaN,NaN,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,NaN,NaN,This Amish pickled beets recipe is a gift from...


## Origin

In [32]:
df
df.head(2)

,Name,Description,Cleaned_recipe,Origin,Restriction,Restrictive Ingredient,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,NaN,NaN,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,NaN,NaN,This Amish pickled beets recipe is a gift from...


In [33]:
columns_to_drop = df.columns[4:6]
df = df.drop(columns=columns_to_drop)
df.head(2)

,Name,Description,Cleaned_recipe,Origin,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,This Amish pickled beets recipe is a gift from...


In [34]:
df.to_csv('Origins.csv', index=False)

### Preprocessing

In [58]:
df = pd.read_csv("Origins.csv")
df.head()

,Name,Description,Cleaned_recipe,Origin,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,This Amish pickled beets recipe is a gift from...
2,Amish Macaroni Salad,This Amish macaroni salad is a colorful and fl...,"cups uncooked elbow macaroni , large hard-...",Amish and Mennonite,This Amish macaroni salad is a colorful and fl...
3,Amish Friendship Bread Starter,Make Amish friendship bread starter and share ...,"ounce package active dry yeast , cup warm...",Amish and Mennonite,Make Amish friendship bread starter and share ...
4,My Amish Friend's Caramel Corn,This caramel corn recipe is the best! It comes...,"quarts plain popped popcorn , cups dry roa...",Amish and Mennonite,This caramel corn recipe is the best! It comes...


In [59]:
# Rename Origin to label
df = df.rename(columns={'Origin': 'label'})

In [60]:
label_list = df['label'].to_list()
label_count = {}

for labels in label_list:
    label_list_split = labels.split(',')
    for label in label_list_split:
        label = label.strip()
        if label in label_count:
            label_count[label] += 1
        else:
            label_count[label] = 1

print(f"Number of labels: {len(label_count)}")
print(label_count)

Number of labels: 67
{'Amish and Mennonite': 57, 'Argentinian': 28, 'Australian and New Zealander': 60, 'Austrian': 21, 'Bangladeshi': 9, 'Belgian': 4, 'Brazilian': 60, 'Cajun and Creole': 59, 'Canadian': 60, 'Chilean': 19, 'Chinese': 63, 'Colombian': 7, 'Cuban': 59, 'Danish': 32, 'Dutch': 22, 'Filipino': 60, 'Finnish': 20, 'French': 60, 'German': 58, 'Greek': 59, 'Indian': 61, 'Indonesian': 25, 'Israeli': 21, 'Italian': 62, 'Jamaican': 40, 'Japanese': 57, 'Jewish': 59, 'Korean': 55, 'Lebanese': 50, 'Malaysian': 24, 'Norwegian': 25, 'Pakistani': 27, 'Persian': 55, 'Peruvian': 43, 'Polish': 57, 'Portuguese': 54, 'Puerto Rican': 57, 'Russian': 60, 'Scandinavian': 60, 'Soul Food': 64, 'South African': 16, 'Southern Recipes': 518, 'Spanish': 61, 'Swedish': 51, 'Swiss': 8, 'Tex-Mex': 54, 'Thai': 59, 'Turkish': 38, 'Vietnamese': 57, 'Australian and New Zealander Recipes': 60, 'Cajun and Creole Recipes': 143, 'Canadian Recipes': 140, 'Chinese Recipes': 330, 'Filipino Recipes': 117, 'French Re

In [ ]:
encode_label_type = { key: idx for idx, (key,value) in enumerate (label_count.items()) }
with open ("label_types_encode.json","w") as fp:
  json.dump(encode_label_type, fp)

In [1]:
import pandas as pd

# Assuming df is your DataFrame and 'Name' is your column
df = pd.read_csv('Origins.csv')

# Characters that are not allowed in Windows filenames
invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']

def clean_name(name):
    # Replace spaces and single quotes with underscores
    name = name.replace(' ', '_').replace("'", "_")
    
    # Remove any characters that are not allowed in Windows filenames
    for char in invalid_chars:
        name = name.replace(char, '')
    
    return name

df['Name'] = df['Name'].apply(clean_name)

print(df)


                                           Name  \
0                         Best_Vinegar_Coleslaw   
1     Pennsylvania_Dutch_Pickled_Beets_and_Eggs   
2                          Amish_Macaroni_Salad   
3                Amish_Friendship_Bread_Starter   
4                My_Amish_Friend_s_Caramel_Corn   
...                                         ...   
5746                                Chicken_Pho   
5747                       Authentic_Oxtail_Pho   
5748                                Pho_Ga_Soup   
5749     Vietnamese_Style_Vegetarian_Curry_Soup   
5750                           Instant_Pot®_Pho   

                                            Description  \
0     This tangy vinegar coleslaw is perfect for tho...   
1     This Amish pickled beets recipe is a gift from...   
2     This Amish macaroni salad is a colorful and fl...   
3     Make Amish friendship bread starter and share ...   
4     This caramel corn recipe is the best! It comes...   
...                              

In [2]:
df.to_csv('Origins.csv', index=False)
df

,Name,Description,Cleaned_recipe,Origin,Body
0,Best_Vinegar_Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania_Dutch_Pickled_Beets_and_Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,This Amish pickled beets recipe is a gift from...
2,Amish_Macaroni_Salad,This Amish macaroni salad is a colorful and fl...,"cups uncooked elbow macaroni , large hard-...",Amish and Mennonite,This Amish macaroni salad is a colorful and fl...
3,Amish_Friendship_Bread_Starter,Make Amish friendship bread starter and share ...,"ounce package active dry yeast , cup warm...",Amish and Mennonite,Make Amish friendship bread starter and share ...
4,My_Amish_Friend_s_Caramel_Corn,This caramel corn recipe is the best! It comes...,"quarts plain popped popcorn , cups dry roa...",Amish and Mennonite,This caramel corn recipe is the best! It comes...
...,...,...,...,...,...
5746,Chicken_Pho,Mom's chicken pho recipe... straight from Viet...,"quarts water , pounds chicken bones , wh...",Vietnamese Recipes,Mom's chicken pho recipe... straight from Viet...
5747,Authentic_Oxtail_Pho,This oxtail pho recipe is authentic. This is t...,"water to cover , pounds beef oxtail, or to...",Vietnamese Recipes,This oxtail pho recipe is authentic. This is t...
5748,Pho_Ga_Soup,Vietnamese chicken noodle soup. After ordering...,"tablespoon vegetable oil , small yellow on...",Vietnamese Recipes,Vietnamese chicken noodle soup. After ordering...
5749,Vietnamese_Style_Vegetarian_Curry_Soup,Vietnamese-style curry powder can be found in ...,"tablespoons vegetable oil , onion, coarsel...",Vietnamese Recipes,Vietnamese-style curry powder can be found in ...


## Restrictive Class

In [39]:
df= pd.read_csv('Dataset(final).csv')
df.head(2)

,Name,Description,Cleaned_recipe,Origin,Unnamed: 4,Non-vegan,Non-Vegetarian,Sea-foods,Haram,Nut-allergies,Non-Pescatarian,pork,Meat,beaf,alcohol,dairy,Restriction,Restrictive Ingredient
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df['Body'] = df['Description'].astype(str) + " " + df['Cleaned_recipe'].astype(str) 
df.head(2)

,Name,Description,Cleaned_recipe,Origin,Unnamed: 4,Non-vegan,Non-Vegetarian,Sea-foods,Haram,Nut-allergies,Non-Pescatarian,pork,Meat,beaf,alcohol,dairy,Restriction,Restrictive Ingredient,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This Amish pickled beets recipe is a gift from...


In [41]:
# Assuming you have a DataFrame named df and you want to drop columns 5 to 17
columns_to_drop = df.columns[4:16]  # This selects columns 5 to 17 (0-based index)
# Use the drop method to remove the specified columns
df = df.drop(columns=columns_to_drop)
df.head(2)

,Name,Description,Cleaned_recipe,Origin,Restriction,Restrictive Ingredient,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,NaN,NaN,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,NaN,NaN,This Amish pickled beets recipe is a gift from...


In [42]:
#drop columns origin and Restricive ingredients
df.drop(['Origin','Restrictive Ingredient'], axis=1, inplace=True)
df.head(2)

,Name,Description,Cleaned_recipe,Restriction,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",NaN,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",NaN,This Amish pickled beets recipe is a gift from...


In [44]:
df.shape

(5751, 5)

In [45]:
df = df[df['Restriction'].notnull()]
df.shape

(4834, 5)

In [6]:
df.to_csv('Restrictions.csv', index=False)

In [5]:
import pandas as pd

# Assuming df is your DataFrame and 'Name' is your column
df = pd.read_csv('Restrictions.csv')

# Characters that are not allowed in Windows filenames
invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']

def clean_name(name):
    # Replace spaces and single quotes with underscores
    name = name.replace(' ', '_').replace("'", "_")
    
    # Remove any characters that are not allowed in Windows filenames
    for char in invalid_chars:
        name = name.replace(char, '')
    
    return name

df['Name'] = df['Name'].apply(clean_name)

print(df)


                                           Name  \
0                         Best_Vinegar_Coleslaw   
1     Pennsylvania_Dutch_Pickled_Beets_and_Eggs   
2                          Amish_Macaroni_Salad   
3                Amish_Friendship_Bread_Starter   
4                My_Amish_Friend_s_Caramel_Corn   
...                                         ...   
5746                                Chicken_Pho   
5747                       Authentic_Oxtail_Pho   
5748                                Pho_Ga_Soup   
5749     Vietnamese_Style_Vegetarian_Curry_Soup   
5750                           Instant_Pot®_Pho   

                                            Description  \
0     This tangy vinegar coleslaw is perfect for tho...   
1     This Amish pickled beets recipe is a gift from...   
2     This Amish macaroni salad is a colorful and fl...   
3     Make Amish friendship bread starter and share ...   
4     This caramel corn recipe is the best! It comes...   
...                              

## Restrictive Ingredients

In [52]:
df= pd.read_csv('Dataset(final).csv')
df['Body'] = df['Description'].astype(str) + " " + df['Cleaned_recipe'].astype(str) 
# Assuming you have a DataFrame named df and you want to drop columns 5 to 17
columns_to_drop = df.columns[4:16]  # This selects columns 5 to 17 (0-based index)
# Use the drop method to remove the specified columns
df = df.drop(columns=columns_to_drop)
df.head(2)

,Name,Description,Cleaned_recipe,Origin,Restriction,Restrictive Ingredient,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",Amish and Mennonite,NaN,NaN,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",Amish and Mennonite,NaN,NaN,This Amish pickled beets recipe is a gift from...


In [53]:
#drop columns origin and Restricive ingredients
df.drop(['Origin','Restriction'], axis=1, inplace=True)
df.head(2)

,Name,Description,Cleaned_recipe,Restrictive Ingredient,Body
0,Best Vinegar Coleslaw,This tangy vinegar coleslaw is perfect for tho...,"large head cabbage, cored and finely shredde...",NaN,This tangy vinegar coleslaw is perfect for tho...
1,Pennsylvania Dutch Pickled Beets and Eggs,This Amish pickled beets recipe is a gift from...,"large eggs , ounce cans whole pickled bee...",NaN,This Amish pickled beets recipe is a gift from...


In [54]:
df.shape

(5751, 5)

In [56]:
df = df[df['Restrictive Ingredient'].notnull()]
df.shape

(4834, 5)

In [57]:
df.to_csv('Ingredients.csv', index=False)

In [7]:
import pandas as pd

# Assuming df is your DataFrame and 'Name' is your column
df = pd.read_csv('Ingredients.csv')

# Characters that are not allowed in Windows filenames
invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']

def clean_name(name):
    # Replace spaces and single quotes with underscores
    name = name.replace(' ', '_').replace("'", "_")
    
    # Remove any characters that are not allowed in Windows filenames
    for char in invalid_chars:
        name = name.replace(char, '')
    
    return name

df['Name'] = df['Name'].apply(clean_name)

print(df)


                                        Name  \
0             Amish_Friendship_Bread_Starter   
1                Pennsylvania_Dutch_Corn_Pie   
2                        Amish_Peanut_Butter   
3                             Amish_Meatloaf   
4                         Amish_Potato_Salad   
...                                      ...   
4829                             Chicken_Pho   
4830                    Authentic_Oxtail_Pho   
4831                             Pho_Ga_Soup   
4832  Vietnamese_Style_Vegetarian_Curry_Soup   
4833                        Instant_Pot®_Pho   

                                            Description  \
0     Make Amish friendship bread starter and share ...   
1     This corn pie is my mom's recipe. We grew up e...   
2     This is a tasty peanut butter spread, sometime...   
3     A recipe for Amish meatloaf topped with bacon ...   
4     This is like the potato salad I grew up with i...   
...                                                 ...   
4829  Mom'